In [1]:
import json
import jsonlines
import os
import multiprocessing as mp
import subprocess, shlex
import simplejson as json
import pandas as pd

from dashboard.data_load import load_mouse_blastp_results, load_esmfold
from dashboard.etl import CACHE_DIR, PROTEIN_TOOLS_PATH, sorf_query

from Bio.Seq import Seq
from pathlib import Path
from tqdm import tqdm

from veliadb import base
from veliadb.base import Orf



2023-12-06 02:34:36.817 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-06 02:34:46.200 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-06 02:34:46.201 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-06 02:34:46.203 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-06 02:34:46.204 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-06 02:34:46.204 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-06 02:34:46.205 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-06 02:34:46.205 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-12-

In [2]:
session = base.Session()

In [3]:
import boto3
import gzip
from io import BytesIO
from Bio import SeqIO

# Initialize a boto3 client for S3
s3_client = boto3.client('s3')
# S3 Bucket and file details
bucket_name = 'velia-annotation-dev'
s3_file_key = 'genomes/hg38/GRCh38.p13.genome.fa.gz'
# Get the file object from S3
file_obj = s3_client.get_object(Bucket=bucket_name, Key=s3_file_key)
# Read the file content
fasta_gzipped = file_obj['Body'].read()
# Decompress the file
with gzip.open(BytesIO(fasta_gzipped), 'rt') as f:
    # Parse the FASTA file
    genome_reference = SeqIO.to_dict(SeqIO.parse(f, "fasta"))

In [15]:
with open('all_vtx_gencode_231205.txt', 'r') as infile:
    ids = [int(i.replace('VTX-', '')) for i in infile.readlines()][0:100]


In [17]:
len(ids)

100

In [5]:
orfs = session.query(Orf).filter(Orf.id.in_(ids)).all()


In [6]:
o = session.query(Orf).filter(Orf.id == 852124).one()


In [7]:
o.start

-1

In [8]:
x = sorf_query.parallel_sorf_query(852124, genome_reference)

In [ ]:
ids = 

In [18]:
def pfunc(i):
    r = sorf_query.parallel_sorf_query(i, genome_reference)
    return r

with open('sorf_table.jsonlines', 'w') as fopen:
    with mp.Pool(16) as ppool:
        for r in tqdm(ppool.imap(pfunc, ids), total=len(ids)):
            fopen.write(json.dumps(r))
            fopen.write('\n')

  0%|          | 0/100 [00:00<?, ?it/s]

(psycopg2.OperationalError) SSL SYSCALL error: EOF detected

[SQL: SELECT orf.id AS orf_id, orf.start AS orf_start, orf."end" AS orf_end, orf.strand AS orf_strand, orf.assembly_id AS orf_assembly_id, orf.block_sizes AS orf_block_sizes, orf.chrom_starts AS orf_chrom_starts, orf.phases AS orf_phases, orf.orf_idx AS orf_orf_idx, orf.orf_idx_str AS orf_orf_idx_str, orf.secondary_orf_id AS orf_secondary_orf_id, orf.benchling_id AS orf_benchling_id, orf.aa_seq AS orf_aa_seq, orf.nt_seq AS orf_nt_seq, orf.ensembl_protein_id AS orf_ensembl_protein_id, orf.refseq_protein_id AS orf_refseq_protein_id, orf.openprot_id AS orf_openprot_id, orf.uniprot_id AS orf_uniprot_id, orf.velia_id AS orf_velia_id, orf.attrs AS orf_attrs 
FROM orf 
WHERE orf.id = %(id_1)s]
[parameters: {'id_1': 851972}]
(Background on this error at: https://sqlalche.me/e/14/e3q8)(psycopg2.OperationalError) SSL SYSCALL error: EOF detected

[SQL: SELECT orf.id AS orf_id, orf.start AS orf_start, orf."end" AS orf_end, orf.strand AS 

/opt/conda/envs/veliadash_updated/lib/python3.11/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


/opt/conda/envs/veliadash_updated/lib/python3.11/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


/opt/conda/envs/veliadash_updated/lib/python3.11/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


...

 36%|███▌      | 36/100 [00:06<00:09,  6.70it/s]

 38%|███▊      | 38/100 [00:07<00:16,  3.78it/s]

/opt/conda/envs/veliadash_updated/lib/python3.11/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


 39%|███▉      | 39/100 [00:08<00:17,  3.42it/s]

100%|██████████| 100/100 [00:17<00:00,  5.72it/s]


In [46]:
sorf_df = load_jsonlines_table('sorf_table.jsonlines', index_col='vtx_id')


100%|██████████| 100/100 [00:00<00:00, 122497.20it/s]


In [47]:
sorf_df

,vtx_id,genscript_id,velia_id,secondary_id,ucsc_track,chr,strand,start,end,chrom_starts,block_sizes,phases,aa,nucl,orf_xrefs,gene_xrefs,transcript_xrefs,protein_xrefs,source,transcripts_exact
vtx_id,,,,,,,,,,,,,,,,,,,,
VTX-0851976,VTX-0851976,U738YHG260-570,U738YHG260-570,U738YHG260-570,chr5:112819114-112821943,chr5,+,112819114,112821943,112819114;112821893,228;51,0;0,MAMTKTLYCWEIPGAVKRLGPGPVQHSTTSFTHSLMTREAGVKSESFIFWNRYALTVKPVGSGRKLMNQAWTRTKIQCQLLLNIRSVLLCVF,ATGGCAATGACAAAGACTCTGTATTGTTGGGAAATTCCCGGGGCAGTAAAGAGGCTCGGGCCAGGGCCAGTGCAGCACTCCACAACATCATTCACTCACAGCCTGATGACAAGAGAGGCAGGCGTGAAATCCGAGTCCTTCATCTTTTGGAACAGATACGCGCTTACTGTGAAACCTGTTGGGAGTGGCAGGAAGC...,"APC/NM_001354898.1/mRNA/CDS, APC/NM_001354899.1/mRNA/CDS, APC/NM_001354906.1/mRNA/CDS, APC/ENST00000508376.6/mRNA/CDS, APC/ENST00000512211.6/mRNA/CDS, APC/NM_001354895.1/mRNA/CDS, APC/NM_001127510...",324;583;APC;APC;ENSG00000134982.17;gene-APC;HGNC:583,ENSG00000283968.1;HGNC:16343;KIR3DP1;ENSE00003725728.1;ENSE00003725728.1;exon-CHS.176211.1;exon:ENST00000610346.4:7;exon:ENST00000615023.4:6;exon:ENST00000615197.4:5;exon:ENST00000616079.4:5;exon:...,"APC/NM_001354898.1/mRNA/CDS, APC/NM_001354899.1/mRNA/CDS, APC/NM_001354906.1/mRNA/CDS, APC/ENST00000508376.6/mRNA/CDS, APC/ENST00000512211.6/mRNA/CDS, APC/NM_001354895.1/mRNA/CDS, APC/NM_001127510...",velia_phase3_nan,[]
VTX-0851977,VTX-0851977,U768PHG260-100,U768PHG260-100,U768PHG260-100,chr4:152346876-152352625,chr4,-,152346876,152352625,152346876;152350042;152352470,196;83;156,1;0;0,MSTLESVTYLPEKGLYCQRLPSSRTHGGTESLKGKNTENMGFYGTLKMIFYKMKRKLDHGSEVRSFSLGKKPCKVSEYTSTTGLVPCSATPTTFGDLRAANGQGQQRRRITSVQPPTGLQEWLKMFQVTYLNIVLEMPHLIAAK,ATGTCCACATTAGAATCTGTGACATACCTACCTGAAAAAGGTTTATATTGTCAGAGACTGCCAAGCAGCCGGACACACGGGGGCACAGAATCACTGAAGGGGAAAAATACAGAAAATATGGGTTTCTACGGCACATTAAAAATGATTTTTTACAAAATGAAAAGAAAGTTGGACCATGGTTCTGAGGTCCGCTCTT...,FBXW7/ENST00000604822.1/ncRNA/-;U768PHG260-100,16712;55294;ENSG00000109670.17;FBXW7;FBXW7;gene-FBXW7;HGNC:16712,ENSG00000230915.1;HGNC:23425;HLA-W;11779;7053;ENSG00000125780.12;gene-TGM3;HGNC:11779;TGM3;TGM3;ENSG00000258352.1;HGNC:54545;IFITM3P6;ENSE00003938740.1;exon:ENST00000689910.1:1;ENSE00001845398.1;e...,FBXW7/ENST00000604822.1/ncRNA/-;NaN;U768PHG260-100,velia_phase3_nan,[ENST00000604822.1]
VTX-0851978,VTX-0851978,U768PHG260-102,Phase 3_112,Phase 3_112,chr6:157148767-157167265,chr6,+,157148767,157167265,157148767;157167040,215;226,0;1,MPSAGMQNRPFPGNMSSMTPSSPGMSQQGGPGMGPPMPTVNRKAQEAAAAVMQAAANSAQSRYATQEHAPGRQGSFPGMNQSGLMASSSPYSQPMNNSSSLMNTQAPPYSMAPAMVNSSAGNLGSSALLSPSLSPLLLLGSTSESA,ATGCCATCAGCTGGGATGCAGAACAGACCATTTCCTGGAAATATGAGCAGCATGACCCCCAGTTCTCCTGGCATGTCTCAGCAGGGAGGGCCAGGAATGGGGCCGCCAATGCCAACTGTGAACCGTAAGGCACAGGAGGCAGCCGCAGCAGTGATGCAGGCTGCTGCGAACTCAGCACAAAGCAGGTACGCCACCC...,ARID1B/ENST00000636426.1/ncRNA/-;Phase 3_112;U768PHG260-102,ARID1B;ENSG00000049618.25;HGNC:18040;18040;57492;ARID1B;gene-ARID1B,18464;392862;ENSG00000215045.9;gene-GRID2IP;GRID2IP;GRID2IP;HGNC:18464;ENSG00000275413;ENSG00000275413.1;ENSG00000254264.2;HGNC:5833;IGKV3OR22-2;ENSG00000237754;ENSG00000237754.1;ENSE00003905524.1...,ARID1B/ENST00000636426.1/ncRNA/-;sORF3112;U768PHG260-102,velia_phase3_nan,[ENST00000636426.1]
VTX-0851979,VTX-0851979,U768PHG260-104,U768PHG260-104,U768PHG260-104,chr12:45837360-45839501,chr12,+,45837360,45839501,45837360;45837498;45839329,58;210;173,0;2;2,MFHTVTKCLMSRDRFLKMRGMEILGNLCKAEDNGVLICEYVDQDSYREIICHLTLPDVLLVISTLEVLYMLTEMGDVACTKIAKVEKSIDMLVCLVSMDIQMFGPDALAAVKLIEHPSSSHQMLSEIRPQAIEQVQTQTHVASAPG,ATGTTTCATACTGTTACAAAATGTCTAATGTCAAGGGATAGATTTTTAAAGATGAGAGGCATGGAAATTTTGGGAAATCTTTGCAAAGCAGAAGATAATGGTGTTTTAATTTGTGAATATGTGGATCAGGATTCCTACAGAGAGATCATTTGTCATCTCACTTTACCTGATGTGCTGCTTGTAATCTCAACACTCG...,ARID2/ENST00000480128.1/ncRNA/-;U768PHG260-104,ARID2;ENSG00000189079.18;HGNC:18037;18037;196528;ARID2;gene-ARID2,ATP2B2-IT1;ENSG00000236999.1;HGNC:41309;ENSG00000254787;ENSG00000254787.1;ENSE00003730962.1;exon-CHS.172863.1;exon:ENST00000613373.1:1;exon-XR_007068542.1-1;ENSE00003782926.1;exon:ENST00000632198....,ARID2/E

In [12]:
assert x['aa'] == Seq(x['nucl']).translate()